<div style="display:flex;">
<img alt="New Relic" style="display:block;height:64px" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/brands/new_relic.png" />
<img alt="SoftButterfly" style="display:block;height:64px;margin-left:auto" src="https://gitlab.com/softbutterfly/open-source/open-source-office/-/raw/master/banners/borderless/softbutterfly.png" />
</div>

# New Relic Playground

## Imports

Python Imports

In [ ]:
import json
import os
from datetime import datetime
from textwrap import dedent

Third-party libraries

In [ ]:
import dotenv
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from requests import Session, request

## Useful definitions

In [ ]:
NULL_CURSOR = json.dumps(None)

In [ ]:
def build_query(query_string, query_params=None):
    if query_params is None:
        query_params = {}

    return dedent(query_string.strip()) % query_params

In [ ]:
def get_response_data(response, key_path=None, action="actor"):
    data = response.json().get("data").get(action)

    if key_path is not None:
        for key in key_path.split(":"):
            if key.isdecimal() and isinstance(data, list):
                data = data[int(key)]
            else:
                data = data.get(key)

    return data

In [ ]:
class NewRelicGqlClient(Session):
    """Client for New Relic GraphQL API."""

    url: str = "https://api.newrelic.com/graphql"

    def __init__(self, new_relic_user_key=None):
        super().__init__()

        self.headers.update(
            {
                "Content-Type": "application/json",
                "API-Key": new_relic_user_key,
            }
        )

    def execute(self, query, variables=None, **kwargs):
        data = json.dumps(
            {
                "query": query,
                "variables": variables,
            },
        )
        return self.post(self.url, data=data, **kwargs)

## Client setup

Load credentials

In [ ]:
env_file = "../.env"

dotenv.load_dotenv(env_file)

NEW_RELIC_USER_KEY = os.environ.get("NEW_RELIC_USER_KEY", None)
NEW_RELIC_ACCOUNT_ID = os.environ.get("NEW_RELIC_ACCOUNT_ID", None)

if NEW_RELIC_USER_KEY is None:
    raise ValueError("Environment variable NEW_RELIC_USER_KEY is not set.")

if NEW_RELIC_ACCOUNT_ID is None:
    raise ValueError("Environment variable NEW_RELIC_ACCOUNT_ID is not set.")

Client instantiation

In [ ]:
newrelic = NewRelicGqlClient(NEW_RELIC_USER_KEY)

Define test query

In [ ]:
query = build_query(
    """
        {
            actor {
                user {
                    email
                    name
                    id
                }
            }
        }
    """
)

Executing query

In [ ]:
response = newrelic.execute(query)

Looking for response data

In [ ]:
print(
    json.dumps(
        get_response_data(
            response,
            action="actor",
        ),
        indent=4,
    )
)

## Playground area

### Get data from NRQL

In [ ]:
query_string = """
{
    actor {
        account(id: %(account_id)d) {
            name
            id
        }
        nrql(
            accounts: %(account_id)d,
            query: "FROM Metric SELECT * WHERE integrationName = 'nri-prometheus'"
        ) {
            results
        }
    }
}

"""

query_params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
account_details = get_response_data(
    response,
    key_path="account",
    action="actor",
)
account_details

In [ ]:
nrql_results = get_response_data(
    response,
    key_path="nrql:results",
    action="actor",
)
len(nrql_results)

In [ ]:
nrql_results[0]

In [ ]:
nrql_results_df = pd.DataFrame(
    data=nrql_results,
)
nrql_results_df

### Creating drop rule

In [ ]:
query_string = """
mutation {
    nrqlDropRulesCreate(
        accountId: %(account_id)d,
        rules: {
            action: DROP_DATA,
            description: "Drop dat from Prometheus integration.",
            nrql: "FROM Metric SELECT * WHERE integrationName = 'nri-prometheus'"
        }
    ) {
        failures {
            error {
                description
                reason
            }
            submitted {
                accountId
                action
                description
                nrql
            }
        }
        successes {
            id
            source
            nrql
        }
    }
}

"""

query_params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
failures = get_response_data(
    response,
    key_path="failures",
    action="nrqlDropRulesCreate",
)
failures

In [ ]:
successes = get_response_data(
    response,
    key_path="successes",
    action="nrqlDropRulesCreate",
)
successes

### Look at ingested data

In [ ]:
query_string = """
{
    actor {
        account(id: %(account_id)d) {
            name
            id
        }
        nrql(
            accounts: %(account_id)d,
            query: "FROM Metric SELECT latest(timestamp) WHERE integrationName = 'nri-prometheus'"
        ) {
            results
        }
    }
}
"""

query_params = {
    "account_id": int(NEW_RELIC_ACCOUNT_ID),
}

query = build_query(
    query_string=query_string,
    query_params=query_params,
)

In [ ]:
response = newrelic.execute(query)

In [ ]:
latest_timestamp = get_response_data(
    response,
    key_path="nrql:results:0:latest.timestamp",
    action="actor",
)
latest_timestamp

In [ ]:
latest_ingest_date = datetime.fromtimestamp(latest_timestamp / 1000)


print(latest_ingest_date)